In [ ]:
import cv2
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from collections import deque
from tiki.mini import TikiMini

# TikiMini 초기화
tiki = TikiMini()

# LED 제어 함수
def clear_leds():
    for i in range(16):
        tiki.set_led(0, i, 0, 0, 0)

def draw_shape(shape, color):
    clear_leds()
    r, g, b = color
    
    if shape == 'X':
        indices = [15, 0, 9, 6, 10, 5, 12, 3]
    elif shape == 'O':
        indices = [15, 8, 7, 0, 12, 11, 4, 3, 14, 13, 1, 2]
    elif shape == '#':
        indices = list(set([8, 9, 10, 11, 7, 6, 5, 4, 14, 9, 6, 1, 13, 10, 5, 2]))
    else:
        return
    
    for i in indices:
        tiki.set_led(0, i, r, g, b)

# QR 코드와 LED 패턴 매핑
qr_mapping = {
    'ID_O_R': ('O', (50, 0, 0)),
    'ID_O_G': ('O', (0, 50, 0)),
    'ID_O_B': ('O', (0, 0, 50)),
    'ID_X_R': ('X', (50, 0, 0)),
    'ID_X_G': ('X', (0, 50, 0)),
    'ID_X_B': ('X', (0, 0, 50)),
    'ID_#_R': ('#', (50, 0, 0)),
    'ID_#_G': ('#', (0, 50, 0)),
    'ID_#_B': ('#', (0, 0, 50)),
}

# CSI 카메라 파이프라인
pipeline = (
    "nvarguscamerasrc ! video/x-raw(memory:NVMM), width=640, height=480, format=NV12, framerate=30/1 ! "
    "nvvidconv ! video/x-raw, format=BGRx ! videoconvert ! video/x-raw, format=BGR ! appsink"
)

cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)

if not cap.isOpened():
    pass
else:
    # 비디오 위젯 생성
    video_widget = widgets.Image(format='jpeg', layout=widgets.Layout(width='640px', height='480px'))
    display(video_widget)
    
    def frame_to_bytes(frame):
        _, buf = cv2.imencode('.jpg', frame, [int(cv2.IMWRITE_JPEG_QUALITY), 40])
        return buf.tobytes()
    
    qr_detector = cv2.QRCodeDetector()
    
    # CLAHE 생성 (명암 개선)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    
    try:
        # 2초 동안 프리뷰 (카메라 워밍업)
        start_time = time.time()
        while time.time() - start_time < 1:
            ret, frame = cap.read()
            if ret:
                frame = cv2.flip(frame, -1)
                video_widget.value = frame_to_bytes(frame)
        
        # QR 인식 시도 (최대 5초 또는 인식될 때까지)
        qr_found = False
        attempt_start = time.time()
        
        while not qr_found and (time.time() - attempt_start) < 5:
            ret, frame = cap.read()
            if not ret:
                continue
            
            frame = cv2.flip(frame, -1)
            
            # 방법 1: 원본 이미지로 시도
            data, bbox, _ = qr_detector.detectAndDecode(frame)
            
            # 방법 2: 실패 시 그레이스케일 + CLAHE로 재시도
            if not data:
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                enhanced = clahe.apply(gray)
                data, bbox, _ = qr_detector.detectAndDecode(enhanced)
            
            # QR 인식 성공
            if data:
                qr_found = True
                
                if data in qr_mapping:
                    shape, color = qr_mapping[data]
                    draw_shape(shape, color)
                
                # QR 코드 시각화
                if bbox is not None:
                    pts = bbox.astype(int).reshape(-1, 2)
                    for i in range(len(pts)):
                        cv2.line(frame, tuple(pts[i]), tuple(pts[(i + 1) % len(pts)]), (0, 255, 0), 2)
                    cv2.putText(frame, data, (pts[0][0], pts[0][1] - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            # 화면 업데이트
            video_widget.value = frame_to_bytes(frame)
        
        cap.release()
    
    except KeyboardInterrupt:
        cap.release()
        clear_leds()